# Inversores

In [1]:
from z3 import *
import numpy as np

Possibilidades:

- Todos os inversores a 0: 1 caso
- Todos os inversores a 1: 1 caso - Única possibilidade para no passo a seguir ficarem os inversores todos a 0
- Um dos inversores a 0: 1 caso
- Um dos inversores a 1: 1 caso
- Dois inversores a 0 (e a 1): 2 casos - zeros seguidos ou alternados

## Variante

In [5]:
def declare(i, num_inv=4):
    state = {j: {"in": Int(f"in_s{i}_i{j}"), "out": Int(f"out_s{i}_i{j}")} for j in range(num_inv)}
    return state

def init(state):
    conds = []
    for i in state:
        conds.append(Or(state[i]["in"] == 0, state[i]["in"] == 1))
        conds.append(Or(state[i]["out"] == 0, state[i]["out"] == 1))
        conds.append(state[i]["out"] == state[(i+1)%len(state)]["in"])
    
    return And(conds)

def invert(inv_atual, inv_ant):
    c1 = inv_atual["out"] == inv_ant["out"]
    c2 = inv_atual["out"] == 1 - inv_ant["in"]
    r = Or(c1, c2)
    
    return r  
    
def trans(sys_atual, sys_ant):
    # Restrições da determinação do output de cada inversor
    conds = []
    for i in sys_atual:
        conds.append(sys_atual[(i+1)%len(sys_atual)]["in"] == sys_atual[i]["out"])
        conds.append(invert(sys_atual[i], sys_ant[i]))
    
    r = And(conds)
    
    return r

In [21]:
def kinduction_always(declare, init, trans, inv, pos, k, prop="variant decrease"):
    solver = Solver()
    trace = {i: declare(i) for i in range(k+1)}
    solver.add(init(trace[0]))
    
    # Testar k casos de base
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
        solver.add(pos(trace[i]))
    solver.add(Or([Not(inv(trace[i])) for i in range(k)]))
        
    if solver.check() == sat:
        print(f"The {prop} breaks down in (at least) one of the initial states.\n")
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
                
        return
    elif solver.check() != unsat:
        return
        
    # Testar caso indutivo
    solver = Solver()
    for i in range(k):
        solver.add(trans(trace[i+1], trace[i]))
        solver.add(inv(trace[i]))
    solver.add(Not(inv(trace[k])))
    
    if solver.check() == sat:
        print(f"The {prop} fails in the inductive state.\n")
        
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
                
        return
    else:
        print(f"The {prop} holds.")

In [22]:
def positive_state(state):
    return Sum([state[i]["out"] for i in state]) > 0

def variant(state):
    return Sum([state[i]["out"] for i in state])

def var_positive(state):
    return variant(state) >= 0

def var_decreases(state, l=2):
    states = {i: declare(-i) for i in range(1, l)}
    
    conds = [trans(states[1], state)]
    for i in range(1, l-1):
        conds.append(trans(states[i+1], states[i]))
        
    c1 = And(conds)
    c2 = Or(variant(states[l-1])<variant(state), variant(states[l-1])==0)
    
    lista = []
    for i in state:
        lista.append(state[i]["out"])
        lista.append(state[i]["in"])
    
    r = ForAll(lista, Implies(c1, c2))
    return r

def var_useful(state):
    return Implies(variant(state)==0, Sum([state[i]["out"] for i in state])==0)

kinduction_always(declare, init, trans, var_decreases, positive_state, 10)

The variant decrease breaks down in (at least) one of the initial states.

0 [0, 0, 1, 0]
1 [1, 1, 1, 0]
2 [1, 1, 1, 0]
3 [1, 0, 0, 0]
4 [1, 0, 1, 1]
5 [0, 0, 1, 1]
6 [0, 0, 1, 1]
7 [0, 0, 1, 1]
8 [0, 0, 1, 1]
9 [0, 0, 1, 1]


In [8]:
def time_evolution(declare, init, trans, k):
    # Criar o solver e o traço de estados temporais
    solver = Solver()
    states = {i: declare(i) for i in range(k)}

    # Inicializar o primeiro estado temporal
    solver.add(init(states[0]))

    # Fazer a transição de estados
    for i in range(1, k):
        solver.add(trans(states[i], states[i-1]))

    if solver.check() == sat:
        m = solver.model()
        
        output = {}
        for i in range(k):
            elems = [elem for elem in m if f"_s{i}_" in str(elem)]
            output[i] = {str(elem): m[elem] for elem in elems if "out" in str(elem)}
        
        for k in output.keys():
            keys = sorted(output[k].keys())
            output[k] = [output[k][key] for key in keys]
            print(k, output[k])
            
    return

time_evolution(declare, init, trans, 10)

0 [1, 1, 1, 1]
1 [0, 0, 1, 1]
2 [0, 0, 1, 1]
3 [0, 1, 1, 1]
4 [0, 1, 1, 1]
5 [0, 1, 1, 0]
6 [1, 1, 0, 0]
7 [1, 1, 0, 0]
8 [1, 0, 0, 1]
9 [0, 0, 1, 1]


In [9]:
def possible_states(declare, init, trans, inv, var, k):
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}
    solver.add(init(trace[0]))
    
    for i in range(k-1):
        solver.add(trans(trace[i+1], trace[i]))
    solver.add(And([inv(trace[i]) for i in range(k)]))
        
    while solver.check() == sat:
        m = solver.model()
        
        outputs, inputs = [], []
        for i in trace[0]:
            outputs.append([m[elem] for elem in m if f"out_s0_i{i}" in str(elem)][0])
            inputs.append([m[elem] for elem in m if f"in_s0_i{i}" in str(elem)][0])
        
        print(f"Possible starting state: {outputs}")
            
        for i in trace[0]:
            solver.add(trace[0][i]["out"] != outputs[i])
            solver.add(trace[0][i]["in"] != inputs[i])
                
    return
    
possible_states(declare, init, trans, var_decreases, variant, 100)

Possible starting state: [0, 0, 0, 0]
Possible starting state: [1, 1, 1, 1]
